In [ ]:
# Model Training:
# 1) Load all data from preprocessing (training/test splits, etc)
# 2) Begin Training Models
    #  a) Decision Tree
    #  b) Naive Bayes
    #  c) Logistic Regression
    #  d) SVM
# 3) Testing Models
# 4) New Iterations

In [26]:
# Imports
from Models import ModelUtil
from Data import Preprocessing
from Visualization import VisualUtil
from Logs import logging as logs
from sklearn.ensemble import AdaBoostClassifier

import importlib
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
import configparser
import numpy as np

config = configparser.ConfigParser()
config.read('Data//config.ini')

importlib.reload(Preprocessing)
importlib.reload(ModelUtil)
importlib.reload(VisualUtil)
importlib.reload(logs)

<module 'Logs.logging' from 'c:\\Users\\Trent\\Desktop\\Senior Design\\shifting_model\\Logs\\logging.py'>

In [27]:
# 1) Load all data from preprocessing 
importlib.reload(Preprocessing)

if("False" in config['DATA']['USE_NEW_PREPROCESSING']):
    infieldDataFrame, outfieldDataFrame = Preprocessing.dataFiltering([], False)
else:
    normDataFrame = Preprocessing.dataProcessing()
    infieldDataFrame, outfieldDataFrame = Preprocessing.dataFiltering(normDataFrame, True)

c:\Users\Trent\Desktop\Senior Design\shifting_model\Data\DataUtil.py:307: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df = df[df["BatterSide"].isin(["Left","Right"])] # 1, 2


In [ ]:
infieldcorrmatrix = infieldDataFrame.corr()
infieldcorrmatrix.to_csv('Infield_Correlation_Matrix')
outfieldcorrmatrix = outfieldDataFrame.corr()
outfieldcorrmatrix.to_csv('Outfield_Correlation_Matrix')

In [34]:
importlib.reload(Preprocessing)

if("False" in config['DATA']['USE_NEW_PREPROCESSING']):
    Y = infieldDataFrame["FieldSlice"]
    X = infieldDataFrame[["PitcherThrows", "BatterSide", "TaggedPitchType", "PlateLocHeight", "PlateLocSide", "ZoneSpeed"]] #, "SpinRate""RelSpeed" "HorzBreak", "VertBreak",, "HorzBreak", "VertBreak"]]
    xTrain, xTest, yTrain, yTest = train_test_split(X, Y, test_size=0.25, random_state=11)
    # adb = AdaBoostClassifier()
    # adb_model = adb.fit(xTrain, yTrain)
else:
    infieldY = infieldDataFrame[0][['Direction','Distance']]
    infieldX = infieldDataFrame[0][infieldDataFrame[1]] 
    if("True" in config['SPLIT']['TTS']):
        xTrain, xTest, yTrain, yTest = train_test_split(infieldX, infieldY, test_size=0.20, random_state=11)
        
    elif("True" in config['SPLIT']['KFold']):
        kf = KFold(n_splits=5, shuffle=True)
        for train_index, test_index in kf.split(infieldX):
            xTrain, xTest = infieldX.iloc[train_index,:], infieldX.iloc[test_index,:]
            yTrain, yTest = infieldY.iloc[train_index,:], infieldY.iloc[test_index,:]

    elif("True" in config['SPLIT']['LOOCV']):
        loo = LeaveOneOut()
        for train_index, test_index in loo.split(infieldX):
            xTrain, xTest = infieldX.iloc[train_index,:], infieldX.iloc[test_index,:]
            yTrain, yTest = infieldY.iloc[train_index,:], infieldY.iloc[test_index,:]

    else:
        print("No Splitting Method Selected")
        

# GroupKFold: (avoids putting data from the same group in the test set -- useful for Pitcher/Batter ID when we implement that.)

In [ ]:
# 2) Training Models

In [32]:
importlib.reload(ModelUtil)
importlib.reload(logs)
# a) Decision Tree
# Need to test these hyperparameters for best case
max_depth = 50
max_features = 30
max_leaf_nodes = 150
result = ModelUtil.runDT(xTrain, yTrain, xTest, yTest, max_depth, max_features, max_leaf_nodes)


training decision tree model...
done!
getting statistics...

logging statistics...
printing statistics...
Model Type: DecisionTree

Training Size = 42213
Testing Size = 14072

Training Accuracy = 0.3615947693838391
Testing Accuracy = 0.3384025014212621

Hyper-Parameters: 

Max Tree Depth: 50
Max Tree Features: 30
Max Leaf Nodes: 150

Accuracy Score for Predicting on Training Data: 0.9372
Accuracy Score for Predicting on Test Data: 0.3384

Overall Average Probabilities
-------------------------------------
Section 1: 17.38%
Section 2: 26.05%
Section 3: 24.53%
Section 4: 19.96%
Section 5: 12.09%

Field Slice Counts for Training Data
--------------------------------------------------
Section	Truth	Prediction
1		7295		6444
2		10967		17875
3		10348		6742
4		8483		8077
5		5120		3075
Amount Correct: 15264
Amount Incorrect: 26949

Field Slice Counts for Testing Data
--------------------------------------------------
Section	Truth	Prediction
1		2513		2139
2		3643		6008
3		3427		2260
4		2881		26

c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:61: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dfTestStats = dftest.groupby(["FieldSliceActual"]).size().reset_index()
c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:65: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dfTestStats["Correct"] = dftemp.groupby(["FieldSliceActual"]).size().reset_index()[0]
c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:67: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain cu

In [33]:
importlib.reload(ModelUtil)
importlib.reload(logs)
# b) Naive Bayes

print(xTrain)

var_smoothing = 1e-9
result = ModelUtil.runNB(xTrain, yTrain, xTest, yTest, var_smoothing)

         PitcherThrows  BatterSide  PlateLocHeight  PlateLocSide  ZoneSpeed
1368973              1           2         1.79593       0.10197   79.35984
173530               2           1         1.99089       0.68989   83.06563
43641                2           2         2.28983       0.57149   87.01479
1593920              2           2         1.84547       1.19446   88.30647
169437               1           1         2.10108       0.66865   72.87832
...                ...         ...             ...           ...        ...
683939               2           2         1.35353       0.39197   75.01937
153093               2           1         3.10934       0.51794   81.41481
456840               2           2         2.17454      -0.27049   85.74510
857652               2           1         2.59162       0.60776   81.06349
210564               1           2         2.41684      -0.10013   83.00322

[42213 rows x 5 columns]
training Naive Bayes model...
done!
getting statistics...
logg

c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:61: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dfTestStats = dftest.groupby(["FieldSliceActual"]).size().reset_index()
c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:65: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dfTestStats["Correct"] = dftemp.groupby(["FieldSliceActual"]).size().reset_index()[0]
c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:67: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain cu

In [35]:
importlib.reload(ModelUtil)
importlib.reload(logs)
# c)Logistic Regression
lr = 0.8
e = 100
result = ModelUtil.runLogReg(xTrain, yTrain, xTest, yTest, lr, e)

training logistic regression model...
done!
getting statistics...
logging statistics...
printing statistics...
Model Type: LogisticRegression

Training Size = 42213
Testing Size = 14072

Training Accuracy = 0.3322673110179329
Testing Accuracy = 0.33889994314951677

Hyper-Parameters: 

Learning Rate: 0.8
Epochs: 100

Accuracy Score for Predicting on Training Data: 0.9272
Accuracy Score for Predicting on Test Data: 0.3389

Overall Average Probabilities
-------------------------------------
Section 1: 17.41%
Section 2: 26.06%
Section 3: 24.55%
Section 4: 19.98%
Section 5: 12.00%

Field Slice Counts for Training Data
--------------------------------------------------
Section	Truth	Prediction
1		7295		2500
2		10967		22578
3		10348		4940
4		8483		11500
5		5120		695
Amount Correct: 14026
Amount Incorrect: 28187

Field Slice Counts for Testing Data
--------------------------------------------------
Section	Truth	Prediction
1		2513		805
2		3643		7684
3		3427		1543
4		2881		3847
5		1608		193
Amo

C:\Users\Trent\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:61: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dfTestStats = dftest.groupby(["FieldSliceActual"]).size().reset_index()
c:\Users

In [36]:
importlib.reload(ModelUtil)
importlib.reload(logs)
# d) SVM
rC = 1
kernel='linear'
degree= 1
gamma= 'scale'
coef0= 0.0
result = ModelUtil.runSVM(xTrain, yTrain, xTest, yTest, rC, kernel, degree, gamma, coef0)

training SVM model...
done!
getting statistics...
logging statistics...


c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:61: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dfTestStats = dftest.groupby(["FieldSliceActual"]).size().reset_index()
c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:65: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dfTestStats["Correct"] = dftemp.groupby(["FieldSliceActual"]).size().reset_index()[0]
c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:67: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain cu

printing statistics...
Model Type: SVM

Training Size = 42213
Testing Size = 14072

Training Accuracy = 0.31333949257337784
Testing Accuracy = 0.32227117680500283

Hyper-Parameters: 

Regularization Constant: 1
Kernel Type: linear
Kernel Degree1
Kernel Coefficient (gamma): scale
Independent Term in Kernel (coef0): 0.0

Accuracy Score for Predicting on Training Data: 0.9562
Accuracy Score for Predicting on Test Data: 0.3223


c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:153: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dfTestStats = dftest.groupby(["FieldSliceActual"]).size().reset_index()
c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:157: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dfTestStats["Correct"] = dftemp.groupby(["FieldSliceActual"]).size().reset_index()[0]
c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:159: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain


Overall Average Probabilities
-------------------------------------
Section 1: 16.98%
Section 2: 26.34%
Section 3: 24.40%
Section 4: 20.21%
Section 5: 12.07%

Field Slice Counts for Training Data
--------------------------------------------------
Section	Truth	Prediction
1		7295		28007.0
2		10967		14206.0
3		10348		nan
4		8483		nan
5		5120		nan
Amount Correct: 13227
Amount Incorrect: 28986

Field Slice Counts for Testing Data
--------------------------------------------------
Section	Truth	Prediction
1		2513		9367.0
2		3643		4705.0
3		3427		nan
4		2881		nan
5		1608		nan
Amount Correct: 4535
Amount Incorrect: 9537
done!


In [ ]:
# z) RandomForestRegressor
for i in range(0, len(trainIn)):
    direction, distance = ModelUtil.runRFR(trainIn[i], trainOut[i], testIn[i], testOut[i])

In [ ]:
# 3) Model Testing:

In [ ]:
# 4) Model Iterations and Improvements

In [ ]:
# 5) Data Visualization

# Temporary method of getting percentages for testing purposes
slices = 5
infieldPercentages  = np.random.dirichlet(np.ones(slices), size=1)[0]
outfieldPercentages = np.random.dirichlet(np.ones(slices), size=1)[0]

VisualUtil.visualizeData(infieldPercentages, outfieldPercentages)
